In [6]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton
import os
import json
from susy_qm import calculate_Hamiltonian

from collections import Counter
import time

In [85]:
potential = 'QHO'
cutoff = 16
shots = 1024

In [86]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

min_3_ev = eigenvalues.argsort()[:4]
#min_eigenvector = np.asarray(eigenvectors[:, min_3_ev[0]])

In [87]:
eigenvalues[min_3_ev]

array([0.+0.j, 1.+0.j, 1.+0.j, 2.+0.j])

In [88]:
#Create operator pool
operator_pool = []
phi = 0.0
for i in range(num_qubits):
    operator_pool.append(qml.RY(phi,wires=[i]))
    #operator_pool.append(qml.RZ(phi,wires=[i]))

c_pool = []

for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                c_pool.append(qml.CRY(phi=phi, wires=[control, target]))

operator_pool = operator_pool + c_pool

In [89]:
def create_circuit(params, op_list, current_basis, pre_basis=[], use_trial=False, trial_op=None, swap=False):

    param_index = 0

    if swap:
        qml.BasisState(pre_basis, wires=range(num_qubits, 2*num_qubits))
        for op in op_list:
            o = type(op)
            if o == qml.CRY:
                w0 = op.wires[0] + num_qubits
                w1 = op.wires[1] + num_qubits
                o(params[param_index], wires=[w0,w1])
                param_index += 1
            else:
                wire = op.wires[0] + num_qubits
                o(params[param_index], wires=wire)
                param_index += 1
    else:
        qml.BasisState(current_basis, wires=range(num_qubits))
        for op in op_list:
            o = type(op)
            o(params[param_index], wires=op.wires)
            param_index +=1

        if use_trial:
            to = type(trial_op)
            to(0.0, wires=trial_op.wires)


In [90]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
@qml.qnode(dev)
def energy_expval(params, op_list, basis_state):

    create_circuit(params, op_list, basis_state)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [91]:
swap_dev = qml.device("default.qubit", wires=2*num_qubits, shots=None)
@qml.qnode(swap_dev)
def swap_test(pre_basis, current_basis, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op):

    create_circuit(prev_params, prev_op_list, current_basis, pre_basis, swap=True)
    create_circuit(op_params, op_list, current_basis, pre_basis, use_trial, trial_op=trial_op)

    qml.Barrier()
    for i in range(num_qubits):
        qml.CNOT(wires=[i, i+num_qubits])    
        qml.Hadamard(wires=i)      

    prob = qml.probs(wires=range(2*num_qubits))

    return prob

In [92]:
def overlap(pre_basis, current_basis, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op):

    probs = swap_test(pre_basis, current_basis, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op)

    overlap = 0
    for idx, p in enumerate(probs):

        bitstring = format(idx, '0{}b'.format(2*num_qubits))

        counter_11 = 0
        for i in range(num_qubits):
            a = int(bitstring[i])
            b = int(bitstring[i+num_qubits])
            if (a == 1 and b == 1):
                counter_11 +=1

        overlap += p*(-1)**counter_11

    return overlap

In [93]:
def multi_swap_test(pre_basis, current_basis, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op=None, num_swap_tests=1):

    results = []
    for _ in range(num_swap_tests):

        ol = overlap(pre_basis, current_basis, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op)
        results.append(ol)
    
    avg_ol = sum(results) / num_swap_tests

    return avg_ol

In [94]:
def loss_f(params, op_list, prev_op_list, prev_params, basis_list, e_level, beta=2.0):

    current_basis = basis_list[e_level]

    energy = energy_expval(params, op_list, current_basis)

    penalty = 0
    pre_level = 0
    if len(prev_op_list) != 0:
        for prev_op, prev_param in zip(prev_op_list, prev_params):
                    pre_basis = basis_list[pre_level]
                    ol = multi_swap_test(pre_basis, current_basis, prev_op, prev_param, op_list, params, use_trial=False)
                    penalty += (beta*ol)
                    pre_level+=1
                    
    return energy + (penalty)

In [95]:
def compute_grad(trial_param, H, num_qubits, trial_op, op_list, op_params, basis_state):

    dev2 = qml.device("default.qubit", wires=num_qubits, shots=None)
    @qml.qnode(dev2)

    def grad_circuit(trial_param, trial_op, op_list, op_params):

        qml.BasisState(basis_state, wires=range(num_qubits))

        param_index = 0
        for op in op_list:
            o = type(op)
            o(op_params[param_index], wires=op.wires)
            param_index +=1

        oph = type(trial_op)
        oph(trial_param, wires=trial_op.wires)

        return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
    
    params = pnp.tensor(trial_param, requires_grad=True)
    grad_fn = qml.grad(grad_circuit)
    grad = grad_fn(params, trial_op, op_list, op_params)
    
    return grad

In [96]:
def grad_plus_overlap(e_level, basis_list, trial_op, trial_param, op_list, op_params, prev_op_list, prev_params, beta=2.0):

    current_basis = basis_list[e_level]

    grad = compute_grad(trial_param, H, num_qubits, trial_op, op_list, op_params, current_basis)
  
    penalty = 0
    pre_level = 0
    if len(prev_op_list) != 0:
            for prev_op, prev_param in zip(prev_op_list, prev_params):
                pre_basis = basis_list[pre_level]
                ol = multi_swap_test(pre_basis, current_basis, prev_op, prev_param, op_list, op_params, use_trial=True, trial_op=trial_op)
                penalty += (beta*ol)
                pre_level+=1

                #print(current_basis, pre_basis)
                #print(f"Applying penalty {penalty} to op {trial_op} with gradient {abs(grad)}")
    
    return abs(grad), penalty

In [103]:
# We need to generate a random seed for each process otherwise each parallelised run will have the same result
seed = (os.getpid() * int(time.time())) % 123456789

# Optimizer
num_energy_levels = 2
num_adapt_steps = 3
num_grad_checks = 10
num_vqe_runs = 1
max_iter = 200
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-2
popsize = 20

beta=1.0

# Main ADAPT-VQE script
prev_op_list = []
prev_params = []
all_energies = []
success_list = []
final_ops_list = []
single_ops = [qml.RY]#,qml.RZ]


#basis_state = [1] + [0]*(num_qubits-1)
#basis_state = [0] + [0]*(num_qubits-1)

#QHO
basis_list = [[1] + [0]*(num_qubits-1),
              [0]*(num_qubits),
              [1] + [0]*(num_qubits-1)
              ]

#AHO
basis_list = [[1] + [0]*(num_qubits-1),
              [1] + [0]*(num_qubits-1),
              [0]*(num_qubits)
              ]

for e_level in range(num_energy_levels):

    #print(f"Running for energy level: {e}")

    op_list = []
    op_params = []
    energies = []
    pool = operator_pool.copy()
    success = False

    current_eigenval = eigenvalues[min_3_ev[e_level]]
    print(f"Looking for energy level {e_level} with eigenvalue {current_eigenval}")

    #if e==1:
        #basis_state = [1] + [0]*(num_qubits-1)
        #basis_state = [0]*(num_qubits)
    #else:
        #basis_state = [0]*(num_qubits)
        #basis_state = [1] + [0]*(num_qubits-1)

    for i in range(num_adapt_steps):

        print(f"Running for adapt step: {i}")

        max_ops_list = []
        
        if i != 0:
            
            pool.remove(most_common_gate)

            if type(most_common_gate) == qml.CRY:
                cq = most_common_gate.wires[0]
                tq = most_common_gate.wires[1]

                for sop in single_ops:
                    if (sop(phi, wires=cq) not in pool):
                        pool.append(sop(phi, wires=cq))

                    if (sop(phi, wires=tq) not in pool):
                        pool.append(sop(phi, wires=tq))
        
        for trial_param in np.random.uniform(phi, phi, size=num_grad_checks):
            grad_list = []
            grads = []
            penalties = []
            for trial_op in pool:
                grad, penalty = grad_plus_overlap(e_level, basis_list, trial_op, trial_param, op_list, op_params, prev_op_list, prev_params, beta)
                grads.append(grad)
                penalties.append(penalty)
                o=type(trial_op)
                grad_op = o(trial_param, wires=trial_op.wires)

                grad_list.append(grad_op)

            grad_norm = np.where(max(grads) != 0, grads / max(grads), 0)
            penalty_norm = np.where(max(penalties) != 0, np.array(penalties) / max(penalties), 0)
            #print(f"grad norm: {grad_norm}")
            #print(f"penalty norm: {grad_norm}")

            gp = grad_norm - penalty_norm
            max_gp = np.argmax(gp)

            max_op = grad_list[max_gp]
            max_ops_list.append(max_op)


        counter = Counter(max_ops_list)
        most_common_gate, count = counter.most_common(1)[0]
        op_list.append(most_common_gate)

        # Generate Halton sequence
        num_dimensions = len(op_list)
        num_samples = popsize
        halton_sampler = Halton(d=num_dimensions, seed=seed)
        halton_samples = halton_sampler.random(n=num_samples)
        scaled_samples = 2 * np.pi * halton_samples

        bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
        x0 = np.concatenate((op_params, np.array([0.0])))
        
        print('Starting VQE')

        res = differential_evolution(loss_f,
                                        bounds=bounds,
                                        args=(op_list, prev_op_list, prev_params, basis_list, e_level, beta),
                                        x0=x0,
                                        maxiter=max_iter,
                                        tol=tol,
                                        atol=abs_tol,
                                        strategy=strategy,
                                        popsize=popsize,
                                        init=scaled_samples,
                                        seed=seed
                                        )
        
        if i!=0: pre_min_e = min_e
        min_e = res.fun
        pre_op_params = op_params.copy()
        op_params = res.x

        

        energies.append(min_e)

        if i!=0:
            if abs(pre_min_e - min_e) < 1e-4:
                print("gradient converged")
                energies.pop()
                op_list.pop()
                final_params = pre_op_params
                success = True
                break
            if abs(current_eigenval-min_e) < 1e-3:
                print("Converged to min e")
                success = True
                final_params = op_params
                break

    if success == False:
        final_params = op_params

    final_ops = []
    #print(op_list)
    for op, param in zip(op_list,final_params):
        dict = {"name": op.name,
                "param": param,
                "wires": op.wires.tolist()
                }
        final_ops.append(dict)

    #print(final_ops)
    prev_op_list.append(op_list)
    prev_params.append(final_params)
    all_energies.append(energies)
    final_ops_list.append(final_ops)
    success_list.append(success)
    

Looking for energy level 0 with eigenvalue 0j
Running for adapt step: 0


c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\pennylane\numpy\tensor.py:155: RuntimeWarning: invalid value encountered in divide
  res = super().__array_ufunc__(ufunc, method, *args, **kwargs)
C:\Users\Johnk\AppData\Local\Temp\ipykernel_7728\2953746925.py:96: RuntimeWarning: invalid value encountered in divide
  penalty_norm = np.where(max(penalties) != 0, np.array(penalties) / max(penalties), 0)


Starting VQE
Running for adapt step: 1
Starting VQE
gradient converged
Looking for energy level 1 with eigenvalue (1+0j)
Running for adapt step: 0
Starting VQE
Running for adapt step: 1
Starting VQE
Running for adapt step: 2
Starting VQE


In [104]:
final_ops_list

[[{'name': 'RY', 'param': np.float64(0.0), 'wires': [0]}],
 [{'name': 'RY', 'param': np.float64(3.68863544232008), 'wires': [0]},
  {'name': 'RY', 'param': np.float64(0.04675128535629591), 'wires': [1]},
  {'name': 'CRY', 'param': np.float64(0.33682430030208543), 'wires': [0, 1]}]]

In [105]:
all_energies

[[np.float64(0.0)],
 [np.float64(0.987993450546211),
  np.float64(0.9701726309626759),
  np.float64(0.9960968477678502)]]

In [106]:
success_list

[True, False]